## imports

In [1]:
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StructField, StringType, DateType, IntegerType, DoubleType
from pyspark.sql.functions import col, explode, array, struct, expr, sum, lit

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1609235520050_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## define bucket

In [2]:
input_bucket = 's3://dh-ud-dend-capstone/stage0_raw/oxford-gov-response'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Oxford government response data

In [3]:
schema = StructType([ \
    StructField("CountryName", StringType(), True), \
    StructField("CountryCode", StringType(), True), \
    StructField("RegionName", StringType(), True), \
    StructField("RegionCode", StringType(), True), \
    StructField("Jurisdiction", StringType(), True), \
    StructField("date", StringType(), True), \
    StructField("C1_School_closing", DoubleType(), True), \
    StructField("C1_Flag", DoubleType(), True), \
    StructField("C2_Workplace_closing", DoubleType(), True), \
    StructField("C2_Flag", DoubleType(), True), \
    StructField("C3_Cancel_public_events", DoubleType(), True), \
    StructField("C3_Flag", DoubleType(), True), \
    StructField("C4_Restrictions_on_gatherings", DoubleType(), True), \
    StructField("C4_Flag", DoubleType(), True), \
    StructField("C5_Close_public_transport", DoubleType(), True), \
    StructField("C5_Flag", DoubleType(), True), \
    StructField("C6_Stay_at_home_requirements", DoubleType(), True), \
    StructField("C6_Flag", DoubleType(), True), \
    StructField("C7_Restrictions_on_internal_movement", DoubleType(), True), \
    StructField("C7_Flag", DoubleType(), True), \
    StructField("C8_International_travel_controls", DoubleType(), True), \
    StructField("E1_Income_support", DoubleType(), True), \
    StructField("E1_Flag", DoubleType(), True), \
    StructField("E2_Debt_contract_relief", DoubleType(), True), \
    StructField("E3_Fiscal_measures", DoubleType(), True), \
    StructField("E4_International_support", DoubleType(), True), \
    StructField("H1_Public_information_campaigns", DoubleType(), True), \
    StructField("H1_Flag", DoubleType(), True), \
    StructField("H2_Testing_policy", DoubleType(), True), \
    StructField("H3_Contact_tracing", DoubleType(), True), \
    StructField("H4_Emergency_investment_in_healthcare", DoubleType(), True), \
    StructField("H5_Investment_in_vaccines", DoubleType(), True), \
    StructField("H6_Facial_Coverings", DoubleType(), True), \
    StructField("H6_Flag", DoubleType(), True), \
    StructField("H7_Vaccination_policy", DoubleType(), True), \
    StructField("H7_Flag", DoubleType(), True), \
    StructField("M1_Wildcard", DoubleType(), True), \
    StructField("ConfirmedCases", DoubleType(), True), \
    StructField("ConfirmedDeaths", DoubleType(), True), \
    StructField("StringencyIndex", DoubleType(), True), \
    StructField("StringencyIndexForDisplay", DoubleType(), True), \
    StructField("StringencyLegacyIndex", DoubleType(), True), \
    StructField("StringencyLegacyIndexForDisplay", DoubleType(), True), \
    StructField("GovernmentResponseIndex", DoubleType(), True), \
    StructField("GovernmentResponseIndexForDisplay", DoubleType(), True), \
    StructField("ContainmentHealthIndex", DoubleType(), True), \
    StructField("ContainmentHealthIndexForDisplay", DoubleType(), True), \
    StructField("EconomicSupportIndex", DoubleType(), True), \
    StructField("EconomicSupportIndexForDisplay", DoubleType(), True), \
  ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
ox_file = '/OxCGRT_latest.csv'
df_ox = spark.read.schema(schema).csv(input_bucket + ox_file,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
datefunc = F.udf(lambda x: datetime.strptime(x, '%Y%m%d'), DateType())
df_ox = df_ox.withColumn('date', datefunc(col('date')))
df_ox = df_ox.withColumn("month",F.month("date"))
df_ox = df_ox.withColumn("year",F.year("date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_ox = df_ox.withColumn("C1_School_closing", df_ox["C1_School_closing"].cast("integer")) \
         .withColumn("C1_Flag", df_ox["C1_Flag"].cast("integer")) \
         .withColumn("C2_Workplace_closing", df_ox["C2_Workplace_closing"].cast("integer")) \
         .withColumn("C2_Flag", df_ox["C2_Flag"].cast("integer")) \
         .withColumn("C3_Cancel_public_events", df_ox["C3_Cancel_public_events"].cast("integer")) \
         .withColumn("C3_Flag", df_ox["C3_Flag"].cast("integer")) \
         .withColumn("C4_Restrictions_on_gatherings", df_ox["C4_Restrictions_on_gatherings"].cast("integer")) \
         .withColumn("C4_Flag", df_ox["C4_Flag"].cast("integer")) \
         .withColumn("C5_Close_public_transport", df_ox["C5_Close_public_transport"].cast("integer")) \
         .withColumn("C5_Flag", df_ox["C5_Flag"].cast("integer")) \
         .withColumn("C6_Stay_at_home_requirements", df_ox["C6_Stay_at_home_requirements"].cast("integer")) \
         .withColumn("C6_Flag", df_ox["C6_Flag"].cast("integer")) \
         .withColumn("C7_Restrictions_on_internal_movement", df_ox["C7_Restrictions_on_internal_movement"].cast("integer")) \
         .withColumn("C7_Flag", df_ox["C7_Flag"].cast("integer")) \
         .withColumn("C8_International_travel_controls", df_ox["C8_International_travel_controls"].cast("integer")) \
         .withColumn("E1_Income_support", df_ox["E1_Income_support"].cast("integer")) \
         .withColumn("E1_Flag", df_ox["E1_Flag"].cast("integer")) \
         .withColumn("E2_Debt_contract_relief", df_ox["E2_Debt_contract_relief"].cast("integer")) \
         .withColumn("H1_Public_information_campaigns", df_ox["H1_Public_information_campaigns"].cast("integer")) \
         .withColumn("H1_Flag", df_ox["H1_Flag"].cast("integer")) \
         .withColumn("H2_Testing_policy", df_ox["H2_Testing_policy"].cast("integer")) \
         .withColumn("H3_Contact_tracing", df_ox["H3_Contact_tracing"].cast("integer")) \
         .withColumn("H6_Facial_Coverings", df_ox["H6_Facial_Coverings"].cast("integer")) \
         .withColumn("H6_Flag", df_ox["H6_Flag"].cast("integer")) \
         .withColumn("H7_Vaccination_policy", df_ox["H7_Vaccination_policy"].cast("integer")) \
         .withColumn("H7_Flag", df_ox["H7_Flag"].cast("integer"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
input_bucket = 's3://dh-ud-dend-capstone/stage0_raw/google/'
df_gl_index = spark.read.format('csv').options(header='true').load(input_bucket + 'index.csv')

google_cc = df_gl_index.select('3166-1-alpha-2', '3166-1-alpha-3').distinct()

df_ox = df_ox.alias('o').join(google_cc.alias('g'), on=[col('o.CountryCode') == col('g.3166-1-alpha-3')], how='left')

df_ox = df_ox.withColumnRenamed("3166-1-alpha-2","country_code")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_ox_reduced = df_ox.select("country_code", \
                 "date", \
                 "month", \
                 "year", \
                 "RegionName", \
                 "RegionCode", \
                 "Jurisdiction", \
                 "C1_School_closing", \
                 "C1_Flag", \
                 "C2_Workplace_closing", \
                 "C2_Flag", \
                 "C3_Cancel_public_events", \
                 "C3_Flag", \
                 "C4_Restrictions_on_gatherings", \
                 "C4_Flag", \
                 "C5_Close_public_transport", \
                 "C5_Flag", \
                 "C6_Stay_at_home_requirements", \
                 "C6_Flag", \
                 "C7_Restrictions_on_internal_movement", \
                 "C7_Flag", \
                 "C8_International_travel_controls", \
                 "E1_Income_support", \
                 "E1_Flag", \
                 "E2_Debt_contract_relief", \
                 "E3_Fiscal_measures", \
                 "E4_International_support", \
                 "H1_Public_information_campaigns", \
                 "H1_Flag", \
                 "H2_Testing_policy", \
                 "H3_Contact_tracing", \
                 "H4_Emergency_investment_in_healthcare", \
                 "H5_Investment_in_vaccines", \
                 "H6_Facial_Coverings", \
                 "H6_Flag", \
                 "H7_Vaccination_policy", \
                 "H7_Flag", \
                 "StringencyIndex", \
                 "GovernmentResponseIndex", \
                 "ContainmentHealthIndex", \
                 "EconomicSupportIndex")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_ox_reduced.write.mode('overwrite').partitionBy("country_code").parquet("s3://dh-ud-dend-capstone/stage1_prepared_and_partitioned/oxford-gov-response/oxfgovresp.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…